In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index-readers-file pymupdf
#!pip install llama-index
!pip install llama-index
!pip install ragas
!pip install datasets
!pip install langchainhub

In [ ]:
import os
import openai

In [ ]:
os.environ['OPENAI_API_KEY'] = ''


In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex
from llama_index.core import PromptTemplate
from IPython.display import Markdown, display

In [ ]:
#!pip install llama-index
!pip install llama-index
!pip install ragas
!pip install datasets

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.evaluation import RetrieverEvaluator
from llama_index.llms.openai import OpenAI

import os
import pandas as pd

from datasets import Dataset

In [2]:
documents = SimpleDirectoryReader("/content/Data").load_data()
#documents

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_str = "ngành Xây dựng và Quản lý Dự án Xây dựng đại học Bách Khoa TPHCM bao gồm những chuyên ngành nào ?"

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=5)
# use this for testing
vector_retriever = index.as_retriever(similarity_top_k=5)

In [ ]:
response = query_engine.query(query_str)
print(str(response))

Kỹ thuật Xây dựng, Xây dựng Dân dụng và Công nghiệp, Kỹ thuật Xây dựng, Công trình Giao thông, Quy hoạch và Quản lý Giao thông, Kỹ thuật Xây dựng Công trình Thủy, Thủy lợi - Thủy điện, Kỹ thuật Xây dựng Công trình Biển, Cảng - Công trình biển; Kỹ thuật Cơ sở Hạ tầng; Cấp thoát nước, Kỹ thuật Trắc địa - Bản đồ, Công nghệ Kỹ thuật Vật liệu Xây dựng.


In [ ]:
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [ ]:
prompts_dict = query_engine.get_prompts()

In [ ]:
# to do this, you need to use the langchain object

from langchain import hub

langchain_prompt = hub.pull("rlm/rag-prompt")

In [ ]:
from llama_index.core.prompts import LangchainPromptTemplate

lc_prompt_tmpl = LangchainPromptTemplate(
    template=langchain_prompt,
    template_var_mappings={"query_str": "question", "context_str": "context"},
)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": lc_prompt_tmpl}
)

In [ ]:
prompts_dict = query_engine.get_prompts()

In [ ]:
!pip install llama-index-llms-langchain

In [ ]:
!pip install llama-index-llms-fireworks

In [ ]:
response = query_engine.query(query_str)
print(str(response))

Ngành Xây dựng và Quản lý Dự án Xây dựng tại Đại học Bách Khoa TPHCM bao gồm các chuyên ngành: Kỹ thuật Xây dựng, Xây dựng Dân dụng và Công nghiệp, Công trình Giao thông, Quy hoạch và Quản lý Giao thông, Kỹ thuật Xây dựng Công trình Thủy, Thủy lợi - Thủy điện, Kỹ thuật Xây dựng Công trình Biển, Cảng - Công trình biển, Kỹ thuật Cơ sở Hạ tầng, Cấp thoát nước, Kỹ thuật Trắc địa - Bản đồ, Công nghệ Kỹ thuật Vật liệu Xây dựng.


In [ ]:
{"query": "<query>", "response": "<output_json>"}

{'query': '<query>', 'response': '<output_json>'}

In [ ]:
from llama_index.core.schema import TextNode

few_shot_nodes = []
for line in open("/content/test3.json", "r"):
    few_shot_nodes.append(TextNode(text=line))

few_shot_index = VectorStoreIndex(few_shot_nodes)
few_shot_retriever = few_shot_index.as_retriever(similarity_top_k=5)

In [ ]:
import json


def few_shot_examples_fn(**kwargs):
    query_str = kwargs["query_str"]
    retrieved_nodes = few_shot_retriever.retrieve(query_str)
    # go through each node, get json object

    result_strs = []
    for n in retrieved_nodes:
        raw_dict = json.loads(n.get_content())
        query = raw_dict["query"]
        response_dict = json.loads(raw_dict["response"])
        result_str = f"""\
Query: {query}
Response: {response_dict}"""
        result_strs.append(result_str)
    return "\n\n".join(result_strs)

In [ ]:
# write prompt template with functions
qa_prompt_tmpl_str = """\
Trả lời câu hỏi sau về thông tin tuyển sinh của các trường Đại học Quốc Gia TP Hồ Chí Minh dựa trên thông tin ngữ cảnh đã cung cấp sau đây
Thông tin ngữ cảnh dưới đây.
---------------------
{context_str}
---------------------

Query: {query_str}.
Trả lời: \
"""

def get_context():
    # Replace this with your actual logic to retrieve relevant context from documents
    relevant_docs = [doc for doc in documents]
    return "\n".join([doc.text for doc in relevant_docs])

qa_prompt_tmpl = PromptTemplate(
    qa_prompt_tmpl_str
)



In [ ]:
citation_query_str = (
    "Thông tin liên hệ của khoa ngôn ngữ Trung đại học khoa học xã hội và nhân văn, đại học quốc gia TPHCM"
)

In [ ]:
print(
    qa_prompt_tmpl.format(
        query_str=citation_query_str,context_str ='test'
    )
)

Trả lời câu hỏi sau về thông tin tuyển sinh của các trường Đại học Quốc Gia TP Hồ Chí Minh dựa trên thông tin ngữ cảnh đã cung cấp sau đây
Thông tin ngữ cảnh dưới đây.
---------------------
test
---------------------

Query: Thông tin liên hệ của khoa ngôn ngữ Trung đại học khoa học xã hội và nhân văn, đại học quốc gia TPHCM.
Trả lời: 


In [ ]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [ ]:
display_prompt_dict(query_engine.get_prompts())

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Trả lời câu hỏi sau về thông tin tuyển sinh của các trường Đại học Quốc Gia TP Hồ Chí Minh dựa trên thông tin ngữ cảnh đã cung cấp sau đây
Thông tin ngữ cảnh dưới đây.
---------------------
{context_str}
---------------------

Query: {query_str}.
Trả lời: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [ ]:
response = query_engine.query(citation_query_str)
print(str(response))

Địa chỉ: Phòng A.305, Trường Đại học Khoa học xã hội và Nhân văn, Đại học Quốc gia thành phố Hồ Chí Minh, 10 -12 Đinh Tiên Hoàng, Phường Bến Nghé, Quận 1, TP. HCM
Điện thoại: (+84)(08) 38293828
Email: nguvantrung@hcmussh.edu.vn


In [ ]:
import pandas as pd

# Specify the file path
file_path = r'/content/Test set (2).xlsx'

# Read the Excel file into a DataFrame
test = pd.read_excel(file_path)

# Print the DataFrame
print(test)

                                            question   ground_truth
0  Mã ngành Địa Lý học chương trình tiêu chuẩn đạ...  [': 7310501']


In [ ]:
questions = test["question"].to_list()

In [ ]:
answers = []
contexts = []

for i in questions:
  response_vector = query_engine.query(i)### ???
  answers.append(response_vector)
  contexts.append([a.get_text() for a in response_vector.source_nodes])

In [ ]:
ground_truths = [[a] for a in test["ground_truth"]]

In [ ]:
for num, i in enumerate(ground_truths):
    if type(i) != str:
        ground_truths[num] = str(i)

In [ ]:
answers = [str(i) for i in answers]

In [ ]:
datasample = {
    "question": questions,
    "contexts": contexts,
    "answer": answers,
    "ground_truth": ground_truths
}

dataset = Dataset.from_dict(datasample)


In [ ]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
#from ragas.metrics.critique import harmfulness

metrics = [
    faithfulness,
]

In [ ]:
from ragas import evaluate
#from ragas.metrics import context_precision, faithfulness, answer_relevancy, context_recall

#result = evaluate(dataset, metrics=metrics)

result = evaluate(dataset, metrics=[faithfulness,
    answer_relevancy,
    context_precision,
    context_recall],raise_exceptions=False)
print(result)


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

No statements were generated from the answer.
{'faithfulness': nan, 'answer_relevancy': 0.8383, 'context_precision': 0.3333, 'context_recall': 0.7500}


/usr/local/lib/python3.10/dist-packages/ragas/evaluation.py:304: RuntimeWarning: Mean of empty slice
  value = np.nanmean(self.scores[cn])


In [2]:
rs = result.to_pandas()
#rs.head()
rs

In [ ]:
rs.to_excel("test.xlsx")